# Compute probabilities of escape for variants in antibody selections

This notebook analyzes the counts of the different in antibody selections relative to a neutralization standard, and computes probabilities of escape (fraction not neutralized).

Import Python modules:

In [ ]:
import os

import Bio.SeqIO

import alignparse.utils

import altair as alt

import dms_variants.codonvarianttable

import pandas as pd

import yaml

In [ ]:
# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

Get configuration information:

In [ ]:
# If you are running notebook interactively rather than in pipeline that handles
# working directories, you may have to first `os.chdir` to appropriate directory.
os.chdir("../test_example")

with open("config.yaml") as f:
    config = yaml.safe_load(f)

## Compute the escape probabilities
First create our codon variant table without any counts:

In [ ]:
geneseq = str(Bio.SeqIO.read(config["gene_sequence_codon"], "fasta").seq)

variants = dms_variants.codonvarianttable.CodonVariantTable(
    barcode_variant_file=config["codon_variants"],
    geneseq=geneseq,
    allowgaps=True,
    substitutions_are_codon=True,
    primary_target="gene",
    substitutions_col="codon_substitutions",
)

Now get the variant counts and add to the codon variant table:

In [ ]:
barcode_runs = pd.read_csv(config["processed_barcode_runs"]).query(
    "exclude_after_counts == 'no'"
)

variant_counts = pd.concat([
    pd.read_csv(
            os.path.join(config["variant_counts_dir"], f"{tup.library_sample}.csv")
    ).assign(library=tup.library, sample=tup.sample)
    for tup in barcode_runs.itertuples()
])

variants.add_sample_counts_df(variant_counts)

Get the antibody selections data frame:

In [ ]:
antibody_selections = pd.read_csv(config["antibody_selections"])

Compute the probabilities escape using [dms_variants.CodonVariantTable.prob_escape](https://jbloomlab.github.io/dms_variants/dms_variants.codonvarianttable.html#dms_variants.codonvarianttable.CodonVariantTable.prob_escape):

In [ ]:
params = {}
for param in ["min_neut_standard_frac", "min_neut_standard_count"]:
    val = config[f"prob_escape_{param}"]
    print(f"Requring {param} of {val}")
    params[param] = val

prob_escape, neut_standard_fracs, neutralization = variants.prob_escape(
    selections_df=antibody_selections, **params,
)

## Fraction of barcode counts for neutralization standard

Make an interactive plot showing this fraction of counts that are neutralization standard for each antibody sample and its no-antibody control.
Note the fractions are plotted on a symlog scale:

In [ ]:
# make tidy version of neut_standard_fracs
melt_cols = ["antibody_frac", "no-antibody_frac"]
neut_standard_fracs_tidy = (
    neut_standard_fracs
    .melt(
        id_vars=[c for c in neut_standard_fracs.columns if c not in melt_cols],
        value_vars=melt_cols,
        value_name="neut_standard_frac",
        var_name="sample_type",
    )
    .assign(
        sample_type=lambda x: x["sample_type"].str.replace("_frac", ""),
        library_sample=lambda x: x["library"] + " " + x["antibody_sample"],
    )
)

# set up selections over other columns of interest
selection_names = ["library", "date", "antibody", "virus_batch"]
selections = [
    alt.selection_point(
        fields=[col],
        bind=alt.binding_select(
            options=[None] + neut_standard_fracs_tidy[col].unique().tolist(),
            labels=["all"] + [str(x) for x in neut_standard_fracs_tidy[col].unique()],
            name=col,
        )
    )
    for col in selection_names
]

neut_standard_fracs_chart = (
    alt.Chart(neut_standard_fracs_tidy)
    .encode(
        x=alt.X(
            "neut_standard_frac",
            title="neutralization standard fraction",
            scale=alt.Scale(type="symlog", constant=0.02, domainMax=1),
        ),
        y=alt.Y("library_sample", title=None),
        color="sample_type",
        shape="sample_type",
        tooltip=[
            alt.Tooltip(c, format=".2g") if c == "neut_standard_frac" else c
            for c in neut_standard_fracs_tidy.columns if not "library_sample" in c
        ]
    )
    .mark_point()
    .properties(width=275, height=alt.Step(14))
    .add_selection(*selections)
    .configure_axis(labelLimit=500)
)
for selection in selections:
    neut_standard_fracs_chart = neut_standard_fracs_chart.transform_filter(selection)

neut_standard_fracs_chart

## Overall fraction neutralization by number of mutations
Plot neutralization (1 - probability of escape) averaged over all variants with a given number of mutations) for just the primary target.
First, get the neutralization data ready to plot by just getting data for primary target, melting, and doing a few other transformations:

In [ ]:
cols_to_melt = ["prob_escape", "prob_escape_uncensored"]
assert "target" not in neutralization.columns, "only implemented for primary target"
neut_to_plot = (
    neutralization
    .melt(
        id_vars=[c for c in neutralization.columns if c not in cols_to_melt],
        var_name="censored",
        value_name="fraction not neutralized",
    )
    .assign(
        library_sample=lambda x: x["library"] + " " + x["antibody_sample"],
        censored=lambda x: (x["censored"] == "prob_escape").map({True: "yes", False: "no"}),
        n_aa_substitutions=lambda x: (
            x["n_aa_substitutions"].map(
                lambda n: f">{n - 1}" if n == x["n_aa_substitutions"].max() else str(n)
            )
        ),
    )
    .rename(columns={c: c.replace("_", " ") for c in neutralization.columns if c.endswith("count")})
    .merge(antibody_selections, validate="many_to_one", how="left")
)

Now make the plot. 
You can mouseover points for details, use the dropdown to subset samples, click on the legend to show only specific number of amino-acid substitutions or only censored (to [0, 1]) values or raw values.
You should get worried if the censored and uncensored values look dramatically different:

In [ ]:
censored_selection = alt.selection_point(
    fields=["censored"],
    bind="legend",
)

n_aa_substitutions_selection = alt.selection_point(
    fields=["n_aa_substitutions"], bind="legend",
)

neutralization_chart = (
    alt.Chart(neut_to_plot)
    .encode(
        x=alt.X("fraction not neutralized"),
        y=alt.Y("library_sample", title=None),
        color=alt.Color(
            "n_aa_substitutions:N",
            title="amino-acid mutations",
            scale=alt.Scale(domain=neut_to_plot["n_aa_substitutions"].unique()),
        ),
        shape=alt.Shape(
            "censored:N",
            title="censored between 0 & 1",
            scale=alt.Scale(
                domain=neut_to_plot["censored"].unique(),
                range=["triangle-up", "triangle-down"],
            ),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g")
            if c.endswith("_count") or c.startswith("fraction") or c == "antibody_concentration"
            else c
            for c in neut_to_plot.columns
            if "library_sample" not in c
        ],
    )
    .mark_point(filled=True, size=50, opacity=0.7)
    .properties(width=300, height=alt.Step(14))
    .add_selection(censored_selection, n_aa_substitutions_selection, *selections)
    .transform_filter(censored_selection)
    .transform_filter(n_aa_substitutions_selection)
    .configure_axis(labelLimit=500)
)
for selection in selections:
    neutralization_chart = neutralization_chart.transform_filter(selection)

neutralization_chart

## Variants with sufficient no-antibody counts
We only calculate the probability of escape for variants with some minimum number of no-antibody counts.

First, compute the threshold number of counts required in the no-antibody control samples.
The minimum no-antibody counts threshold is just a number, but the minimum fraction of no-antibody counts threshold converts to a different number for each sample:

In [ ]:
min_no_antibody_frac = config["prob_escape_min_no_antibody_frac"]
min_no_antibody_counts = config["prob_escape_min_no_antibody_counts"]
print(f"Filtering with thresholds {min_no_antibody_frac=} and {min_no_antibody_counts=}")

min_no_antibody_count_threshold_df = (
    prob_escape
    .groupby(["library", "no-antibody_sample"], as_index=False)
    .aggregate(total_no_antibody_count=pd.NamedAgg("no-antibody_count", "sum"))
    .assign(
        threshold_count=min_no_antibody_counts,
        threshold_frac=lambda x: x["total_no_antibody_count"] * min_no_antibody_frac,
        threshold=lambda x: x[["threshold_count", "threshold_frac"]].max(axis=1).round().astype(int),
    )
)

min_no_antibody_count_threshold_df

Now draw a boxplot for the no-antibody samples with a black line at the median, boxes spanning the 25th to 75th percentiles, lines spanning the minimum to the maximum, and a red line indciating the threshold.
You can mouseover bars for details and use the dropdown selections to just show certain subsets.
Note that y-axis uses a symlog scale:

In [ ]:
# get data to plot
no_antibody_count_boxplot_df = (
    prob_escape
    [["library", "no-antibody_sample", "no-antibody_count", "barcode"]]
    .drop_duplicates()
    .merge(
        min_no_antibody_count_threshold_df.drop(columns="total_no_antibody_count"),
        validate="many_to_one",
    )
    .assign(
        variant_above_threshold=lambda x: (x["no-antibody_count"] >= x["threshold"]).astype(int),
        count_above_threshold=lambda x: x["no-antibody_count"] * x["variant_above_threshold"],
    )
    .groupby(["library", "no-antibody_sample", "threshold"], as_index=False)
    .aggregate(
        median=pd.NamedAgg("no-antibody_count", "median"),
        percentile_25=pd.NamedAgg("no-antibody_count", lambda s: s.quantile(0.25)),
        percentile_75=pd.NamedAgg("no-antibody_count", lambda s: s.quantile(0.75)),
        min=pd.NamedAgg("no-antibody_count", "min"),
        max=pd.NamedAgg("no-antibody_count", "max"),
        nvariants=pd.NamedAgg("barcode", "count"),
        total_counts=pd.NamedAgg("no-antibody_count", "sum"),
        variants_above_threshold=pd.NamedAgg("variant_above_threshold", "sum"),
        counts_above_threshold=pd.NamedAgg("count_above_threshold", "sum"),
    )
    .assign(
        library_sample=lambda x: x["library"] + " " + x["no-antibody_sample"],
        frac_counts_above_threshold=lambda x: x["counts_above_threshold"] / x["total_counts"],
        frac_variants_above_threshold=lambda x: x["variants_above_threshold"] / x["nvariants"],
    )
    .drop(columns=["counts_above_threshold", "variants_above_threshold", "total_counts", "nvariants"])
    .merge(
        antibody_selections[["no-antibody_sample", "virus_batch", "date", "library"]].drop_duplicates(),
        how="left",
        validate="one_to_one",
    )
)
assert len(no_antibody_count_boxplot_df) == no_antibody_count_boxplot_df["library_sample"].nunique()

# make plot
no_antibody_count_base = (
    alt.Chart(no_antibody_count_boxplot_df)
    .encode(
        y=alt.Y("library_sample", title=None),
        tooltip=[
            alt.Tooltip(c, format=".2g") if no_antibody_count_boxplot_df[c].dtype == float else c
            for c in no_antibody_count_boxplot_df.columns if c != "library_sample"
        ],
    )
)

no_antibody_count_quartile_bars = (
    no_antibody_count_base
    .encode(
        alt.X(
            "percentile_25",
            scale=alt.Scale(type="symlog", constant=20),
            title="counts for variant",
        ),
        alt.X2("percentile_75"),
    )
    .mark_bar(color="blue")
)

no_antibody_count_range_lines = (
    no_antibody_count_base
    .encode(
        alt.X("min"),
        alt.X2("max"),
    )
    .mark_rule(color="blue", opacity=0.5)
)

no_antibody_count_median_lines = (
    no_antibody_count_base
    .encode(alt.X("median"), alt.X2("median"))
    .mark_bar(xOffset=1, x2Offset=-1, color="black")
)

no_antibody_count_threshold = (
    no_antibody_count_base
    .encode(alt.X("threshold"), alt.X2("threshold"))
    .mark_bar(xOffset=1, x2Offset=-1, color="red")
)

no_antibody_count_chart = (
    no_antibody_count_quartile_bars
    + no_antibody_count_range_lines
    + no_antibody_count_median_lines
    + no_antibody_count_threshold
).configure_axis(labelLimit=500).properties(width=350, height=alt.Step(14))

for s, name in zip(selections, selection_names):
    if name != "antibody":
        no_antibody_count_chart = no_antibody_count_chart.add_selection(s).transform_filter(s)

no_antibody_count_chart

Plot the fraction of all variants, and fraction of all counts, that are above the thresholds for the no-antibody conditions:

In [ ]:
frac_counts_df = no_antibody_count_boxplot_df.drop(
    columns=["median", "threshold", "min", "max", "percentile_25", "percentile_75"],
).rename(columns={"frac_counts_above_threshold": "counts", "frac_variants_above_threshold": "variants"})

fraction_type_selection = alt.selection_point(
    fields=["fraction_type"], bind="legend",
)

frac_counts_chart = (
    alt.Chart(frac_counts_df)
    .transform_fold(fold=["counts", "variants"], as_=["fraction_type", "fraction"])
    .encode(
        y=alt.Y("library_sample", title=None),
        tooltip=[
            alt.Tooltip(c, format=".2g") if frac_counts_df[c].dtype == float else c
            for c in frac_counts_df.columns if c != "library_sample"
        ],
        x=alt.X(
            "fraction:Q",
            title="fraction above threshold",
            scale=alt.Scale(domain=(0, 1)),
        ),
        color=alt.Color(
            "fraction_type:N",
            title=None,
            scale=alt.Scale(domain=["counts", "variants"]),
        ),
    )
    .mark_point(filled=True, size=50)
    .properties(width=200, height=alt.Step(14))
    .configure_axis(labelLimit=500)
    .add_selection(fraction_type_selection)
    .transform_filter(fraction_type_selection)
)

for s, name in zip(selections, selection_names):
    if name != "antibody":
        frac_counts_chart = frac_counts_chart.add_selection(s).transform_filter(s)

frac_counts_chart

Now just get probability of escape measurements for variants that exceed no-antibody count threshold:

In [ ]:
prob_escape_filtered = (
    prob_escape
    .merge(
        min_no_antibody_count_threshold_df[["library", "no-antibody_sample", "threshold"]],
        validate="many_to_one",
        how="left",
    )
    .query("`no-antibody_count` >= threshold")
)

## Distribution of probability escape across variants
Plot distribution of prob escape values across non-filtered variants.
The boxes span 25th to 75th percentile, the black vertical line is at the median, and the thin lines extend from the min to max.
You can show either the censored (to between 0 and 1) or uncensored probabilities of escape.
Note the plot uses a symlog scale:

In [ ]:
# get data to plot
prob_escape_boxplot_df = (
    prob_escape_filtered
    .melt(
        id_vars=["library", "antibody_sample", "no-antibody_sample"],
        value_vars=["prob_escape", "prob_escape_uncensored"],
        var_name="censored",
        value_name="probability escape",
    )
    .groupby(
        ["library", "antibody_sample", "no-antibody_sample", "censored"],
        as_index=False,
    )
    .aggregate(
        median=pd.NamedAgg("probability escape", "median"),
        percentile_25=pd.NamedAgg("probability escape", lambda s: s.quantile(0.25)),
        percentile_75=pd.NamedAgg("probability escape", lambda s: s.quantile(0.75)),
        min=pd.NamedAgg("probability escape", "min"),
        max=pd.NamedAgg("probability escape", "max"),
    )
    .assign(censored=lambda x: (x["censored"] == "prob_escape"))
    .merge(
        antibody_selections[["antibody_sample", "antibody_library_sample", "virus_batch", "date", "library", "antibody"]],
        how="left",
        on=["library", "antibody_sample"],
        validate="many_to_one",
    )
)
assert len(prob_escape_boxplot_df) == 2 * prob_escape_boxplot_df["antibody_library_sample"].nunique()

# make plot
prob_escape_base = (
    alt.Chart(prob_escape_boxplot_df)
    .encode(
        y=alt.Y("antibody_library_sample", title=None),
        tooltip=[
            alt.Tooltip(c, format=".3g")
            if prob_escape_boxplot_df[c].dtype == "float"
            else c
            for c in prob_escape_boxplot_df.columns if c != "antibody_library_sample"
        ],
    )
)

prob_escape_quartile_bars = (
    prob_escape_base
    .encode(
        alt.X(
            "percentile_25",
            scale=alt.Scale(type="symlog", constant=0.25),
            title="probability escape",
        ),
        alt.X2("percentile_75"),
    )
    .mark_bar(color="blue")
)

prob_escape_range_lines = (
    prob_escape_base
    .encode(
        alt.X("min"),
        alt.X2("max"),
    )
    .mark_rule(color="blue", opacity=0.5)
)

prob_escape_median_lines = (
    prob_escape_base
    .encode(alt.X("median"), alt.X2("median"))
    .mark_bar(xOffset=1, x2Offset=-1, color="black")
)

prob_escape_chart = (
    prob_escape_quartile_bars
    + prob_escape_range_lines
    + prob_escape_median_lines
).configure_axis(labelLimit=500).properties(width=350, height=alt.Step(14))

censored_dropdown_selection = alt.selection_point(
    fields=["censored"],
    value=[{"censored": True}],
    bind=alt.binding_select(
        options=prob_escape_boxplot_df["censored"].unique().tolist(),
        name="censored",
    )
)

for s in list(selections) + [censored_dropdown_selection]:
    prob_escape_chart = prob_escape_chart.add_selection(s).transform_filter(s)

prob_escape_chart

## Correlations in variant-level escape probabilities
Analyze correlations of escape probabilities of different variants in same library:

In [ ]:
assert (
    len(prob_escape_filtered.groupby(["library", "antibody_sample", "no-antibody_sample"]))
    == len(prob_escape_filtered.groupby(["library", "antibody_sample"]))
)

prob_escape_tidy = (
    prob_escape_filtered
    .rename(columns={"prob_escape": "censored", "prob_escape_uncensored": "uncensored"})
    .melt(
        id_vars=["antibody_sample", "barcode", "library"],
        value_vars=["censored", "uncensored"],
        value_name="prob_escape",
        var_name="censored",
    )
    .assign(censored=lambda x: x["censored"].map({"censored": True, "uncensored": False}))
)

corrs = (
    dms_variants.utils.tidy_to_corr(
        df=prob_escape_tidy,
        sample_col="antibody_sample",
        label_col="barcode",
        value_col="prob_escape",
        group_cols=["library", "censored"],
    )
    .assign(r2=lambda x: x["correlation"]**2)
    .drop(columns="correlation")
)

# add other properties
suffixes = ["_1", "_2"]
for suffix in suffixes:
    corrs = (
        corrs.merge(
            antibody_selections,
            left_on=["library", f"antibody_sample{suffix}"],
            right_on=["library", "antibody_sample"],
            validate="many_to_one",
            suffixes=suffixes,
        )
        .drop(columns=["antibody_library_sample", "no-antibody_library_sample", "no-antibody_sample", "antibody_sample", "replicate"])
    )

# make columns identical for both samples drop suffix for selections
for col in antibody_selections.columns:
    if f"{col}_1" in corrs.columns and f"{col}_2" in corrs.columns and col != "antibody_sample":
        corrs = (
            corrs
            .assign(
                equal=lambda x: x[f"{col}_1"] == x[f"{col}_2"],
                **{col: lambda x: x[f"{col}_1"].where(x["equal"], pd.NA)},
            )
            .drop(columns=[f"{col}{suffix}" for suffix in suffixes])
        )

In [ ]:
for library, library_corr in corrs.groupby("library"):
    corr_chart = (
        alt.Chart(library_corr)
        .encode(
            alt.X("antibody_sample_1", title=None),
            alt.Y("antibody_sample_2", title=None),
            color=alt.Color("r2", scale=alt.Scale(zero=True)),
            tooltip=[
                alt.Tooltip(c, format=".3g") if c == "r2" else c
                for c in ["library", "antibody_sample_1", "antibody_sample_2", "r2"]
            ],
        )
        .mark_rect(stroke="black")
        .properties(width=alt.Step(15), height=alt.Step(15), title=library)
        .configure_axis(labelLimit=500)
    )
    for s, name in list(zip(selections, selection_names)) + [(censored_dropdown_selection, "censored")]:
        if name != "library":
            corr_chart = corr_chart.add_selection(s).transform_filter(s)
    display(corr_chart)

## Convert to reference-based numbering
Re-number the `aa_substitutions` column from sequential to reference numbering; also keep the sequential numbering in a column called `aa_substitutions_sequential`.
We also drop a few unneeded columns:

In [ ]:
renumber = alignparse.utils.MutationRenumber(
    number_mapping=pd.read_csv(config["site_numbering_map"]),
    old_num_col="sequential_site",
    new_num_col="reference_site",
    wt_nt_col=None,
)

renumbered_prob_escape = (
    prob_escape_filtered
    .drop(columns=["threshold", "codon_substitutions", "n_codon_substitutions"])
    .rename(columns={"aa_substitutions": "aa_substitutions_sequential"})
    .assign(
        aa_substitutions=lambda x: (
            x["aa_substitutions_sequential"].apply(
                renumber.renumber_muts,
                allow_gaps=True,
                allow_stop=True,
            )
        ),
    )
)

renumbered_prob_escape.head()